In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import col, datediff, countDistinct, sum, max, lit

In [0]:
spark = SparkSession.builder.appName("RFM_Kmeans").getOrCreate()
spark.catalog.setCurrentCatalog("dev")
spark.catalog.setCurrentDatabase("ecommerce_db_sv")

In [0]:
df_orders = spark.sql("""
    SELECT 
        o.order_id,
        c.customer_unique_id,
        o.order_purchase_timestamp,
        o.order_status,
        p.payment_value
    FROM orders_sv o
    INNER JOIN customer_sv c ON o.customer_id = c.customer_id
    INNER JOIN order_payment_sv p ON o.order_id = p.order_id
    WHERE o.order_status = 'delivered'
""")
df_orders.show(5, truncate=True)

In [0]:
max_date = df_orders.agg(max("order_purchase_timestamp")).collect()[0][0]

rfm = df_orders.groupBy("customer_unique_id").agg(
    datediff(lit(max_date), max("order_purchase_timestamp")).alias("Recency"),
    countDistinct("order_id").alias("Frequency"),
    sum("payment_value").alias("Monetary")
)
rfm.show(10)

In [0]:
# Định dạng Vector
assembler = VectorAssembler(
    inputCols=["Recency", "Frequency", "Monetary"],
    outputCol="rfm_features"
)
rfm_vector = assembler.transform(rfm)

# Chuẩn hóa với StandardScaler
scaler = StandardScaler(inputCol="rfm_features", outputCol="scaledFeatures", withStd=True, withMean=True)
scaler_model = scaler.fit(rfm_vector)
rfm_scaled = scaler_model.transform(rfm_vector)

rfm_scaled.select("Recency", "Frequency", "Monetary","rfm_features", "scaledFeatures").show(5, truncate=False)

In [0]:
kmeans = KMeans(featuresCol="rfm_features", predictionCol="cluster", k=3, seed=42)
model = kmeans.fit(rfm_scaled)
rfm_kmeans = model.transform(rfm_scaled)
rfm_kmeans.select("customer_unique_id", "Recency", "Frequency", "Monetary", "rfm_features", "cluster").show(10)

In [0]:
rfm_kmeans.groupBy("cluster").agg(
    {
        "Recency": "mean",
        "Frequency": "mean",
        "Monetary": "mean"}
).orderBy("cluster").show()

### Nhận xét:
1. **Recency (Thời gian mua hàng gần nhất)**:
   - Cụm **0** có Recency thấp nhất (**127.59**), tức là nhóm này có xu hướng mua hàng gần đây hơn so với các cụm khác.
   - Cụm **1** có giá trị Recency cao nhất (**386.62**), cho thấy đây là nhóm khách hàng ít quay lại mua hàng trong thời gian gần đây.

2. **Monetary (Giá trị chi tiêu trung bình)**:
   - Điểm đáng chú ý là cụm **2** có giá trị Monetary rất cao (**1046.45**), cao hơn hẳn so với hai cụm còn lại (**135.26** và **131.74**). Điều này có thể cho thấy nhóm khách hàng này chi tiêu rất mạnh tay khi mua hàng, dù Recency ở mức trung bình.

3. **Frequency (Tần suất mua hàng)**:
   - Nhìn chung, tất cả các cụm đều có Frequency trung bình gần **1**, nhưng cụm **2** có mức **1.12**, cao hơn một chút so với các cụm khác. Điều này có thể chỉ ra rằng nhóm khách hàng này có xu hướng mua hàng hơn một lần.

4. **Tổng kết**:
   - **Nhóm khách hàng ổn định (Cụm 0):** Đây có thể là khách hàng trung bình.
   - **Nhóm khách hàng có Recency cao (Cụm 1):** Đây là những khách hàng ít quay lại.
   - **Nhóm khách hàng chi tiêu cao (Cụm 2 ):** Đây có thể là khách hàng VIP hoặc khách hàng có giá trị cao. 

### Gợi ý chiến lược:
- **Chăm sóc nhóm khách hàng chi tiêu cao (cụm 2)**: Đây có thể là khách hàng VIP hoặc có giá trị cao. Có thể tạo chương trình ưu đãi hoặc dịch vụ riêng để giữ chân nhóm này.
- **Tăng tần suất mua hàng**: Vì Frequency vẫn khá thấp, có thể xem xét chiến lược kích cầu mua sắm, chẳng hạn như giảm giá cho lần mua kế tiếp.
- **Tìm hiểu nguyên nhân nhóm khách hàng ít quay lại (cụm 1)**: Có thể khảo sát hoặc phân tích lý do họ không mua hàng thường xuyên để có chiến lược cải thiện.
